In [ ]:
# Instalação das dependências
!pip install geopandas
!pip install networkx pyvis

from IPython.display import IFrame, display
from itertools import combinations
from pyvis.network import Network
from google.colab import drive
from pathlib import Path

import geopandas as gpd
import unicodedata, re
import networkx as nx
import pandas as pd
import numpy as np
import sys

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.1 MB/s eta 0:00:00


In [ ]:
# Conexão com o google drive
try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    sys.exit()

PASTA_PROJETO = '/content/drive/MyDrive/Projeto Grafos/'
CAMINHO_BAIRROS = f'{PASTA_PROJETO}bairros.geojson'
CAMINHO_VIAS = f'{PASTA_PROJETO}trechos-de-logradouros.geojson'
CAMINHO_SAIDA_CSV = f'{PASTA_PROJETO}grafo_bairros_com_vias.csv'
ARQUIVO_HTML_SAIDA = f'{PASTA_PROJETO}grafo_via_principal_interativo.html'

BAIRRO_NOME_COL = 'EBAIRRNOMEOF' # Coluna que tem o nome oficial do bairro
VIA_NOME_COL = 'NLGPAVOFIC'      # Coluna que tem o nome oficial da rua
VIA_ID_COL = 'CLOGRACODI'        # Coluna que tem o código único da rua

CRS_ORIGINAL = 'EPSG:4326'
CRS_METRICO = 'EPSG:31985'
BUFFER_METROS = 5

Mounted at /content/drive


In [ ]:
# Leitura dos arquivos geojson
try:
    gdf_bairros = gpd.read_file(CAMINHO_BAIRROS)
    gdf_vias = gpd.read_file(CAMINHO_VIAS)
except Exception as e:
    sys.exit()

# Reprojeção (de grau para metros)
gdf_bairros = gdf_bairros.to_crs(CRS_METRICO)
gdf_vias = gdf_vias.to_crs(CRS_METRICO)

# "Engrossamento" das vias
gdf_vias['geometry'] = gdf_vias.buffer(BUFFER_METROS)

# União dos datasets. Resultado: Tabela com cada rua e quais bairros ela toca
vias_com_bairros = gpd.sjoin(gdf_vias, gdf_bairros, how='inner', predicate='intersects')

Carregando os arquivos de mapa (GeoJSON)...
Arquivos carregados com sucesso!
Total de bairros encontrados: 94
Total de trechos de ruas encontrados: 10226
Convertendo os mapas de 'graus' para 'metros' (EPSG:EPSG:31985)...
Criando 'zona de busca' de 5m ao redor de cada rua...
Ruas 'engrossadas' com sucesso.
Iniciando a Junção Espacial (Spatial Join)...
Junção Espacial finalizada. Agora temos uma tabela de ruas com os bairros que elas tocam.


In [ ]:
lista_de_arestas = [] # Aresta = (bairro_origem, bairro_destino, Nome_da_Rua)

agrupado_por_rua = vias_com_bairros.groupby(VIA_ID_COL)
# ID | Rua            | Bairro
# 1  | Av. Caxangá    | Cordeiro
# 2  | Av. Caxangá    | Iputinga

for cod_logradouro, grupo in agrupado_por_rua:
    bairros_tocados = grupo[BAIRRO_NOME_COL].unique()

    # Se uma rua toca em mais de um bairro, pode ser uma aresta
    if len(bairros_tocados) > 1:
        nome_da_rua = grupo[VIA_NOME_COL].iloc[0]

        if not isinstance(nome_da_rua, str):
            continue

        if nome_da_rua.startswith('Rio'):
            continue

        for bairro_origem, bairro_destino in combinations(bairros_tocados, 2):

            par_ordenado = tuple(sorted((bairro_origem, bairro_destino)))

            lista_de_arestas.append((par_ordenado[0], par_ordenado[1], nome_da_rua))

if not lista_de_arestas:
    sys.exit()

# Transforma a lista em uma tabela do Pandas
df_final = pd.DataFrame(lista_de_arestas, columns=['bairro_origem', 'bairro_destino', 'logradouro'])

df_grafo_final = df_final.drop_duplicates().sort_values(by=['bairro_origem', 'bairro_destino', 'logradouro'])

df_grafo_final.to_csv(CAMINHO_SAIDA_CSV, index=False, encoding='utf-8')

Processando as fronteiras para achar os pares...
Processamento de fronteiras finalizado.


In [ ]:
# O arquivo CSV criado no script anterior
arquivo_csv_completo = f'{PASTA_PROJETO}grafo_bairros_com_vias.csv'

try:
    dados_completos = pd.read_csv(arquivo_csv_completo).dropna(subset=['logradouro'])
except Exception as e:
    sys.exit()

if dados_completos.empty:
    sys.exit()

Abrindo o arquivo CSV: /content/drive/MyDrive/Projeto Grafos/grafo_bairros_com_vias.csv
Beleza, arquivo carregado. Agora vou filtrar pela 'via principal'...


In [ ]:
# Filtragem
# Quanto menor o número, maior a prioridade.
mapa_prioridade = {
    'AVENIDA': 1,
    'ESTRADA': 2,
    'PONTE': 3,
    'VIA': 4,
    'RODOVIA': 5,
    'RUA': 6,
    'TRAVESSA': 7,
    'VIELA': 8,
    'BECO': 9
}

dados_completos['Tipo_Via'] = dados_completos['logradouro'].str.split().str[0].str.upper().str.replace(':', '')

dados_completos['Prioridade'] = dados_completos['Tipo_Via'].map(mapa_prioridade).fillna(99)

# Ordena primeiro por bairro e depois pela prioridade, fazendo com que a via principal de cada par apareça primeiro
dados_ordenados = dados_completos.sort_values(by=['bairro_origem', 'bairro_destino', 'Prioridade'])

dados_via_principal = dados_ordenados.drop_duplicates(subset=['bairro_origem', 'bairro_destino'], keep='first')

# Gera outro CSV com as vias principais apenas
dados_via_principal.to_csv(f'{PASTA_PROJETO}grafo_via_principal.csv', index=False, encoding='utf-8')

Processamento feito! Reduzimos para 782 conexões principais.


In [ ]:
# Foi reduzido de 782 para 244 linhas removendo as conexões de bairros não adjascentes através de um processo MANUAL
# Esses dados tratados foram colocados em um csv que teve upload no drive
dados_via_principal_tratados = pd.read_csv(f'{PASTA_PROJETO}bairros_com_vias_principais_tratados.csv')

DADOS_LOGRADOUROS = 'facequadra.csv'
dados_logradouros = PASTA_PROJETO + DADOS_LOGRADOUROS

try:
    dados_completos = pd.read_csv(dados_logradouros, sep=';', engine='python')
except Exception as e:
    sys.exit()

dados_completos.drop(columns=['codguiasesarjetas', 'descguiasesarjetas','codredesgoto', 'codlimpezaurbana', 'codredeagua', 'codarborizacao', 'codcoleta', 'cod_emplacamento', 'codpavimentacao', 'codiluminacao', 'codredetelefonica', 'descredetelefonica', 'setor', 'face', 'dsqf', 'distrito', 'quadra', 'codlogradouro', 'cep', 'codbairro', 'valor_v0', 'codgaleriapluviais', 'descgaleriapluviais', 'codredeeletrica', 'descredeeletrica', 'municipio', 'uf'], inplace=True)

Arquivo carregado com sucesso!
Index(['nomebairro', 'desciluminacao', 'descpavimentacao', 'desc_emplacamento',
       'desccoleta', 'descarborizacao', 'descredeagua', 'desclimpezaurbana',
       'descredesgoto', 'nome_oficial_logradouro', 'nome_logradouro_completo',
       'nome_logradouro_concatenado'],
      dtype='object')


In [ ]:
def normalize_logradouro(s): # Essa função é usada para normalizar os nomes das ruas
    if pd.isna(s):
        return ""
    s = str(s).strip()
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")
    s = re.sub(r"\s+", " ", s).upper()
    s = re.sub(r"[.,()/:\\\-;\"']", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Aplica normalização se ainda não existir
if 'logradouro_norm' not in dados_via_principal_tratados.columns:
    dados_via_principal_tratados['logradouro_norm'] = dados_via_principal_tratados['logradouro'].apply(normalize_logradouro)

if 'logradouro_norm' not in dados_completos.columns:
    dados_completos['logradouro_norm'] = dados_completos['nome_oficial_logradouro'].apply(normalize_logradouro)

# Escolha as colunas de atributos que irão ser unidas
cols_attr = ['descpavimentacao', 'descarborizacao', 'desccoleta', 'descredeagua', 'desclimpezaurbana', 'descredesgoto', 'desciluminacao', 'desc_emplacamento']

dados_face_lookup = dados_completos.drop_duplicates(subset=['logradouro_norm'], keep='first').set_index('logradouro_norm')[cols_attr]

# Mapeia cada coluna de atributos para o dataframe menor (244 linhas) - onde a união ocorre
for c in cols_attr:
    dados_via_principal_tratados[c] = dados_via_principal_tratados['logradouro_norm'].map(dados_face_lookup[c])

dados_via_principal_tratados[cols_attr] = dados_via_principal_tratados[cols_attr].fillna('')

dados_via_principal_tratados.to_csv(f'{PASTA_PROJETO}grafo_via_principal_com_atributos.csv', index=False, encoding='utf-8')

Linhas do edges: 244
  descpavimentacao: 33 faltantes
  descarborizacao: 33 faltantes
  desccoleta: 33 faltantes
  descredeagua: 33 faltantes
  desclimpezaurbana: 33 faltantes
  descredesgoto: 33 faltantes
  desciluminacao: 33 faltantes
  desc_emplacamento: 33 faltantes

Amostra com atributos (primeiras 8 linhas):


,bairro_origem,bairro_destino,logradouro,Tipo_Via,Prioridade,logradouro_norm,descpavimentacao,descarborizacao,desccoleta,descredeagua,desclimpezaurbana,descredesgoto,desciluminacao,desc_emplacamento
0,Aflitos,Encruzilhada,Avenida João de Barros,AVENIDA,1,AVENIDA JOAO DE BARROS,ASFALTO,COM ARBORIZACAO,CONVENCIONAL ALTERNADA COM SELETIVA,COM REDE DE AGUA,REGULAR DIARIA,COM REDE DE ESGOTO,VAPOR MERCURIO,SEM EMPLACAMENTO
1,Aflitos,Espinheiro,Avenida Conselheiro Rosa e Silva,AVENIDA,1,AVENIDA CONSELHEIRO ROSA E SILVA,ASFALTO,COM ARBORIZACAO,CONVENCIONAL ALTERNADA COM SELETIVA,COM REDE DE AGUA,REGULAR DIARIA,COM REDE DE ESGOTO,VAPOR MERCURIO,SEM EMPLACAMENTO
2,Aflitos,Graças,Avenida Conselheiro Rosa e Silva,AVENIDA,1,AVENIDA CONSELHEIRO ROSA E SILVA,ASFALTO,COM ARBORIZACAO,CONVENCIONAL ALTERNADA COM SELETIVA,COM REDE DE AGUA,REGULAR DIARIA,COM REDE DE ESGOTO,VAPOR MERCURIO,SEM EMPLACAMENTO
3,Aflitos,Rosarinho,Avenida Santos Dumont,AVENIDA,1,AVENIDA SANTOS DUMONT,PARALELO,COM ARBORIZACAO,CONVENCIONAL ALTERNADA COM SELETIVA,COM REDE DE AGUA,REGULAR ALTERNADA,COM REDE DE ESGOTO,VAPOR MERCURIO,COM EMPLACAMENTO
4,Afogados,Bongi,Estrada do Bongi Armando da Fonte,ESTRADA,2,ESTRADA DO BONGI ARMANDO DA FONTE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Afogados,Cabanga,Avenida Sul Gov. Cid Sampaio,AVENIDA,1,AVENIDA SUL GOV CID SAMPAIO,SEM PAVIMENTACAO,SEM ARBORIZACAO,CONVENCIONAL ALTERNADA COM SELETIVA,COM REDE DE AGUA,REGULAR DIARIA,SEM REDE DE ESGOTO,VAPOR MERCURIO,SEM EMPLACAMENTO
6,Afogados,Ilha Joana Bezerra,Avenida Central,AVENIDA,1,AVENIDA CENTRAL,ASFALTO,SEM ARBORIZACAO,CONVENCIONAL DIARIA COM SELETIVA,COM REDE DE AGUA,REGULAR ALTERNADA,SEM REDE DE ESGOTO,VAPOR MERCURIO,SEM EMPLACAMENTO
7,Afogados,Ilha do Retiro,Estrada dos Remédios,ESTRADA,2,ESTRADA DOS REMEDIOS,CONCRETO,SEM ARBORIZACAO,CONVENCIONAL ALTERNADA COM SELETIVA,COM REDE DE AGUA,REGULAR ALTERNADA,COM REDE DE ESGOTO,VAPOR MERCURIO,SEM EMPLACAMENTO


In [ ]:
# Após análise do csv gerado e preenchimento manual de informações de algumas vias, um novo arquivo mais completo foi gerado
dados_via_principal_tratados_preenchido = pd.read_csv(f'{PASTA_PROJETO}grafo_via_principal_com_atributos_preenchidos.csv')

# Definição dos parâmetros dos pesos
min_weight = 0.5

base_by_tipo = {
    'AVENIDA': 1.0,
    'VIADUTO' : 1.1,
    'PONTE'   : 1.2,
    'ESTRADA' : 1.4,
    'RUA'     : 2.0
}

# Penalidades / benefícios
pen_iluminacao = {
    'FLUORESCENTE': -0.05,
    'COMUM': 0.00,
    'VAPOR MERCURIO': 0.00,
    'SEM ILUMINACAO': 0.30,
    '': 0.0
}

pen_pavimentacao = {
    'CONCRETO': -0.10,
    'ASFALTO': -0.05,
    'PARALELO': 0.10,
    'POLIEDRO': 0.20,
    'OUTROS': 0.10,
    'ESCADARIA': 0.50,
    'SEM PAVIMENTACAO': 0.70,
    '': 0.0
}

pen_coleta = {
    'CONVENCIONAL DIARIA COM SELETIVA': -0.05,
    'CONVENCIONAL DIARIA SEM SELETIVA': 0.00,
    'CONVENCIONAL ALTERNADA COM SELETIVA': 0.00,
    'CONVENCIONAL ALTERNADA SEM SELETIVA': 0.05,
    'MANUAL DIARIA': 0.00,
    'MANUAL ALTERNADA': 0.05,
    'SEM COLETA DE LIXO': 0.50,
    '': 0.0
}

pen_limpeza = {
    'REGULAR DIARIA': -0.05,
    'REGULAR ALTERNADA': 0.00,
    'PROGRAMADA SEMANAL': 0.05,
    'PROGRAMADA MENSAL': 0.15,
    'SEM LIMPEZA PUBLICA': 0.50,
    '': 0.0
}

pen_arborizacao = {
    'COM ARBORIZACAO': -0.05,
    'SEM ARBORIZACAO': 0.05,
    '': 0.0
}

pen_redeagua = {
    'COM REDE DE AGUA': 0.0,
    'SEM REDE DE AGUA': 0.20,
    '': 0.0
}

pen_esgoto = {
    'COM REDE DE ESGOTO': 0.0,
    'SEM REDE DE ESGOTO': 0.30,
    '': 0.0
}

pen_emplacamento = {
    'COM EMPLACAMENTO': -0.05,
    'SEM EMPLACAMENTO': 0.05,
    '': 0.0
}

In [ ]:
# Cálculo dos pesos
dados_via_principal_tratados_preenchido['Tipo_Via'] = dados_via_principal_tratados_preenchido['Tipo_Via'].astype(str).str.upper().str.replace('.', '', regex=False).str.strip()

dados_via_principal_tratados_preenchido['base_tipo'] = dados_via_principal_tratados_preenchido['Tipo_Via'].map(base_by_tipo)

# Aplica fallback (RUA) onde não mapeou
dados_via_principal_tratados_preenchido['base_tipo'] = dados_via_principal_tratados_preenchido['base_tipo'].fillna(base_by_tipo['RUA'])

def penalty_lookup(val, mapping):
    if pd.isna(val):
        return mapping.get('', 0.0)
    key = str(val).strip().upper()
    return mapping.get(key, mapping.get('', 0.0))

dados = dados_via_principal_tratados_preenchido  # Alias curto

dados['pen_pav']   = dados['descpavimentacao'].apply(lambda v: penalty_lookup(v, pen_pavimentacao))
dados['pen_ilum']  = dados['desciluminacao'].apply(lambda v: penalty_lookup(v, pen_iluminacao))
dados['pen_coleta']= dados['desccoleta'].apply(lambda v: penalty_lookup(v, pen_coleta))
dados['pen_arbor'] = dados['descarborizacao'].apply(lambda v: penalty_lookup(v, pen_arborizacao))
dados['pen_agua']  = dados['descredeagua'].apply(lambda v: penalty_lookup(v, pen_redeagua))
dados['pen_esgoto']= dados['descredesgoto'].apply(lambda v: penalty_lookup(v, pen_esgoto))
dados['pen_limpeza']=dados['desclimpezaurbana'].apply(lambda v: penalty_lookup(v, pen_limpeza))
dados['pen_emplac']= dados['desc_emplacamento'].apply(lambda v: penalty_lookup(v, pen_emplacamento))

penalty_cols = ['pen_pav','pen_ilum','pen_coleta','pen_arbor','pen_agua','pen_esgoto','pen_limpeza','pen_emplac']
dados['sum_penalties'] = dados[penalty_cols].sum(axis=1)

,logradouro,Tipo_Via,pen_pav,pen_ilum,pen_coleta,pen_arbor,pen_agua,pen_esgoto,pen_limpeza,pen_emplac,sum_penalties
0,Avenida João de Barros,AVENIDA,-0.05,0.0,0.00,-0.05,0.0,0.0,-0.05,0.05,-0.10
1,Avenida Conselheiro Rosa e Silva,AVENIDA,-0.05,0.0,0.00,-0.05,0.0,0.0,-0.05,0.05,-0.10
2,Avenida Conselheiro Rosa e Silva,AVENIDA,-0.05,0.0,0.00,-0.05,0.0,0.0,-0.05,0.05,-0.10
3,Avenida Santos Dumont,AVENIDA,0.10,0.0,0.00,-0.05,0.0,0.0,0.00,-0.05,0.00
4,Estrada do Bongi Armando da Fonte,ESTRADA,0.70,0.0,0.00,0.05,0.0,0.3,0.15,0.05,1.25
5,Avenida Sul Gov. Cid Sampaio,AVENIDA,0.70,0.0,0.00,0.05,0.0,0.3,-0.05,0.05,1.05
6,Avenida Central,AVENIDA,-0.05,0.0,-0.05,0.05,0.0,0.3,0.00,0.05,0.30
7,Estrada dos Remédios,ESTRADA,-0.10,0.0,0.00,0.05,0.0,0.0,0.00,0.05,0.00


In [ ]:
output_csv = f'{PASTA_PROJETO}adjacencias_bairros.csv'

# Cálculo e atribuição do peso
dados['peso_raw'] = dados['base_tipo'] + dados['sum_penalties']
dados['peso'] = dados['peso_raw'].apply(lambda x: max(x, min_weight)).round(2)

def build_obs_compact(r): # Função para construir a observação
    tipo = str(r.get('Tipo_Via','') or '').strip()
    pav  = str(r.get('descpavimentacao','') or '').strip()
    ilum = str(r.get('desciluminacao','') or '').strip()
    coleta = str(r.get('desccoleta','') or '').strip()
    arb = str(r.get('descarborizacao','') or '').strip()
    agua = str(r.get('descredeagua','') or '').strip()
    esgoto = str(r.get('descredesgoto','') or '').strip()
    limpeza = str(r.get('desclimpezaurbana','') or '').strip()
    emplac = str(r.get('desc_emplacamento','') or '').strip()

    parts = [
        f"tipo={tipo or 'NA'}",
        f"pav={pav or 'NA'}",
        f"ilum={ilum or 'NA'}",
        f"coleta={coleta or 'NA'}",
        f"peso_calc={r.get('peso'):.2f}"
    ]

    missing = []

    if not pav: missing.append('pav')
    if not ilum: missing.append('ilum')
    if not coleta: missing.append('coleta')
    if not arb: missing.append('arb')
    if not agua: missing.append('agua')
    if not esgoto: missing.append('esgoto')
    if not limpeza: missing.append('limpeza')
    if not emplac: missing.append('emplac')
    if missing:
        parts.append(f"missing={','.join(missing)}")

    return ";".join(parts)

# Montagem da observação
dados['observacao'] = dados.apply(build_obs_compact, axis=1)

df_out = dados[['bairro_origem','bairro_destino','logradouro','observacao','peso']].copy()
df_out.to_csv(output_csv, index=False, encoding='utf-8-sig')

Peso — estatísticas (peso):
count    244.000000
mean       1.493443
std        0.671141
min        0.750000
25%        0.950000
50%        1.300000
75%        2.050000
max        3.500000

Amostra salva em: /content/drive/MyDrive/Projeto Grafos/adjacencias_bairros.csv



,bairro_origem,bairro_destino,logradouro,observacao,peso
0,Aflitos,Encruzilhada,Avenida João de Barros,tipo=AVENIDA;pav=ASFALTO;ilum=VAPOR MERCURIO;c...,0.90
1,Aflitos,Espinheiro,Avenida Conselheiro Rosa e Silva,tipo=AVENIDA;pav=ASFALTO;ilum=VAPOR MERCURIO;c...,0.90
2,Aflitos,Graças,Avenida Conselheiro Rosa e Silva,tipo=AVENIDA;pav=ASFALTO;ilum=VAPOR MERCURIO;c...,0.90
3,Aflitos,Rosarinho,Avenida Santos Dumont,tipo=AVENIDA;pav=PARALELO;ilum=VAPOR MERCURIO;...,1.00
4,Afogados,Bongi,Estrada do Bongi Armando da Fonte,tipo=ESTRADA;pav=SEM PAVIMENTACAO;ilum=VAPOR M...,2.65
5,Afogados,Cabanga,Avenida Sul Gov. Cid Sampaio,tipo=AVENIDA;pav=SEM PAVIMENTACAO;ilum=VAPOR M...,2.05
6,Afogados,Ilha Joana Bezerra,Avenida Central,tipo=AVENIDA;pav=ASFALTO;ilum=VAPOR MERCURIO;c...,1.30
7,Afogados,Ilha do Retiro,Estrada dos Remédios,tipo=ESTRADA;pav=CONCRETO;ilum=VAPOR MERCURIO;...,1.40
8,Afogados,Imbiribeira,Avenida Central,tipo=AVENIDA;pav=ASFALTO;ilum=VAPOR MERCURIO;c...,1.30
9,Afogados,Jiquiá,Avenida Central,tipo=AVENIDA;pav=ASFALTO;ilum=VAPOR MERCURIO;c...,1.30


In [ ]:
arquivo_adj = f'{PASTA_PROJETO}adjacencias_bairros.csv'

adj_df = pd.read_csv(arquivo_adj)

# Função para converter peso para float
def safe_float(x, fallback=1.0):
    try:
        return float(x)
    except Exception:
        return fallback

adj_df['peso'] = adj_df['peso'].apply(lambda v: safe_float(v, 1.0))

grafo_visual = Network(
    height='900px',
    width='100%',
    heading='',
    bgcolor='#f0f0f0',
    font_color='black'
)

grafo_visual.toggle_physics(True)
# grafo_visual.show_buttons(filter_=['physics', 'nodes', 'edges'])

lista_de_bairros = pd.concat([adj_df['bairro_origem'], adj_df['bairro_destino']]).unique()

for bairro in lista_de_bairros:
    grafo_visual.add_node(
        bairro,
        label=str(bairro),
        color='#c92a2a'
    )

for _, linha in adj_df.iterrows():
    bairro_o = linha['bairro_origem']
    bairro_d = linha['bairro_destino']
    via_principal = linha['logradouro']
    peso = safe_float(linha.get('peso', 1.0), 1.0)

    titulo_aresta = f"{via_principal}</n><b>Peso:</b> {peso}"

    # Adiciona aresta com espessura controlada por 'value'
    grafo_visual.add_edge(
        bairro_o,
        bairro_d,
        title=titulo_aresta,
        value=peso if peso > 0 else 1.0,
        font={'size': 8}
    )

Path(ARQUIVO_HTML_SAIDA).parent.mkdir(parents=True, exist_ok=True)
grafo_visual.save_graph(ARQUIVO_HTML_SAIDA)